In [1]:
import openai_setup
import os
from langchain_openai import AzureChatOpenAI

AZURE_OPENAI_ENDPOINT:https://anildwaopenaiwestus.openai.azure.com/


In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import format_document

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [4]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS

llm = AzureChatOpenAI(
    
    deployment_name=os.environ["DEPLOYMENT_NAME"]
)

store = LocalFileStore("./cache/")
embeddings_from_model = AzureOpenAIEmbeddings(azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"])

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_from_model, store, namespace=embeddings_from_model.model
)

try: 
    faiss_db = FAISS.load_local("faiss_index", cached_embedder)
except(Exception) as e:
    print("Local index not found, please create FAISS index using rag builder notebook.")  

In [5]:
java_code = """
import com.paypal.sdk.core.nvp.*;
import com.paypal.sdk.services.NVPCallerServices;

public class Checkout {

    public static void main(String[] args) {
        try {
            NVPCallerServices caller = new NVPCallerServices();
            NVPProfile profile = new NVPProfile();
            
            // Set up your API credentials and environment
            profile.setAPIUsername("YOUR_API_USERNAME");
            profile.setAPIPassword("YOUR_API_PASSWORD");
            profile.setSignature("YOUR_API_SIGNATURE");
            profile.setEnvironment("sandbox"); // Use "live" for production
            
            caller.setProfile(profile);
            
            // Create a new API request
            NVPDecoder decoder = new NVPDecoder();
            NVPEncoder encoder = new NVPEncoder();
            encoder.add("METHOD", "SetExpressCheckout");
            encoder.add("VERSION", "204"); // Specify the version of the API you're using
            encoder.add("PAYMENTREQUEST_0_AMT", "10.00"); // Set checkout amount
            encoder.add("PAYMENTREQUEST_0_CURRENCYCODE", "USD"); // Set currency
            encoder.add("RETURNURL", "http://yourwebsite.com/success"); // Set return URL after payment
            encoder.add("CANCELURL", "http://yourwebsite.com/cancel"); // Set cancel URL
            
            // Execute the API operation
            String strNVPRequest = encoder.encode();
            String strNVPResponse = caller.call(strNVPRequest);
            
            // Decode the response
            decoder.decode(strNVPResponse);
            
            if ("Success".equalsIgnoreCase(decoder.get("ACK"))) {
                System.out.println("Checkout initiated successfully.");
                // Extract token and redirect user for payment approval
                String token = decoder.get("TOKEN");
                // Redirect to PayPal for approval
                String redirectURL = "https://www.sandbox.paypal.com/cgi-bin/webscr?cmd=_express-checkout&token=" + token;
                System.out.println("Redirect user to: " + redirectURL);
            } else {
                System.out.println("Error: " + decoder.get("L_LONGMESSAGE0")); // Print error message
            }
            
        } catch (Exception e) {
            e.printStackTrace();
        }
    }
}

"""

In [6]:
query = f"convert the following java code to use paypal rest api instead of NVP api. full java code here: {java_code}"


In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import  AzureOpenAIEmbeddings
from IPython.display import Markdown
from langchain.schema import AIMessage, HumanMessage, SystemMessage


retriever = faiss_db.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
system_prompt_template = "You are paypal code migration assistant that converts paypal NVP code to Orders V2 REST API"

system_message_template = ChatPromptTemplate.from_messages([
                ("system", system_prompt_template),
                ("human", "Hello, how are you doing?"),
                ("ai", "I'm doing well, thanks!")

])
initial_prompt = SystemMessage(content=system_prompt_template)
new_prompt = (
    initial_prompt + HumanMessage(content="Hello, how are you doing?") + AIMessage(content="I'm doing well, thanks!")
)


In [11]:
retrieval_chain = (
    {"context": retriever |  _combine_documents, "question": RunnablePassthrough()}
    | new_prompt + prompt
    | llm
    | StrOutputParser()
)

result = retrieval_chain.invoke(query)


# Display the Markdown content
Markdown(result)

To migrate the given Java code from using PayPal's NVP/SOAP API to the REST API, you'll need to use the PayPal REST SDK or make direct HTTPS calls to the REST API endpoints. Below is an example of how you might convert the code using direct HTTPS calls with the PayPal REST API.

Please note that the following code is a simplified illustration. In a production environment, you'd want to handle exceptions more gracefully, store your access token securely, and ensure that you adhere to best practices for managing sensitive information, such as API credentials.

```java
import org.json.JSONObject;
import java.io.OutputStream;
import java.net.HttpURLConnection;
import java.net.URL;
import java.util.Base64;

public class Checkout {

    private static final String CLIENT_ID = "YOUR_CLIENT_ID";
    private static final String CLIENT_SECRET = "YOUR_CLIENT_SECRET";
    private static final String SANDBOX_API_BASE_URL = "https://api.sandbox.paypal.com";

    public static void main(String[] args) {
        try {
            // Get an access token from PayPal
            String accessToken = getAccessToken();

            // Create a new PayPal order
            JSONObject order = new JSONObject();
            order.put("intent", "CAPTURE");

            // Construct application context with the return and cancel URLs
            JSONObject applicationContext = new JSONObject();
            applicationContext.put("return_url", "http://yourwebsite.com/success");
            applicationContext.put("cancel_url", "http://yourwebsite.com/cancel");
            order.put("application_context", applicationContext);

            // Construct a purchase unit with the amount and currency
            JSONObject purchaseUnit = new JSONObject();
            JSONObject amount = new JSONObject();
            amount.put("currency_code", "USD");
            amount.put("value", "10.00");
            purchaseUnit.put("amount", amount);
            order.put("purchase_units", new JSONObject[] { purchaseUnit });

            // Create order request
            URL url = new URL(SANDBOX_API_BASE_URL + "/v2/checkout/orders");
            HttpURLConnection conn = (HttpURLConnection) url.openConnection();
            conn.setRequestMethod("POST");
            conn.setRequestProperty("Content-Type", "application/json");
            conn.setRequestProperty("Authorization", "Bearer " + accessToken);
            conn.setDoOutput(true);

            try (OutputStream os = conn.getOutputStream()) {
                byte[] input = order.toString().getBytes("utf-8");
                os.write(input, 0, input.length);
            }

            int responseCode = conn.getResponseCode();
            if (responseCode == HttpURLConnection.HTTP_CREATED) {
                // If the order is created successfully
                System.out.println("Checkout initiated successfully.");

                // Extract the approval URL to redirect the user
                JSONObject jsonResponse = new JSONObject(new String(conn.getInputStream().readAllBytes()));
                String approvalUrl = jsonResponse.getJSONArray("links").getJSONObject(1).getString("href");

                System.out.println("Redirect user to: " + approvalUrl);
            } else {
                // Handle errors
                System.out.println("Error: " + conn.getResponseMessage());
            }
        } catch (Exception e) {
            e.printStackTrace();
        }
    }

    private static String getAccessToken() throws Exception {
        URL url = new URL(SANDBOX_API_BASE_URL + "/v1/oauth2/token");
        HttpURLConnection conn = (HttpURLConnection) url.openConnection();
        conn.setRequestMethod("POST");
        conn.setRequestProperty("Accept-Language", "en_US");
        conn.setRequestProperty("Content-Type", "application/x-www-form-urlencoded");
        String credentials = CLIENT_ID + ":" + CLIENT_SECRET;
        String encodedCredentials = Base64.getEncoder().encodeToString(credentials.getBytes());
        conn.setRequestProperty("Authorization", "Basic " + encodedCredentials);
        conn.setDoOutput(true);

        try (OutputStream os = conn.getOutputStream()) {
            byte[] input = "grant_type=client_credentials".getBytes("utf-8");
            os.write(input, 0, input.length);
        }

        int responseCode = conn.getResponseCode();
        if (responseCode == HttpURLConnection.HTTP_OK) {
            JSONObject jsonResponse = new JSONObject(new String(conn.getInputStream().readAllBytes()));
            return jsonResponse.getString("access_token");
        } else {
            throw new RuntimeException("Failed to get access token: " + conn.getResponseMessage());
        }
    }
}
```

This code does the following:

1. Retrieves an access token from PayPal.
2. Constructs a new order object with the intent to capture the payment immediately, the purchase unit with the amount and currency, and application context with return and cancel URLs.
3. Sends a POST request to the PayPal REST API to create a new order.
4. If successful, retrieves the approval URL from the response, which the user can visit to approve the payment.

Remember to replace `YOUR_CLIENT_ID` and `YOUR_CLIENT_SECRET` with your actual PayPal REST API credentials. Also, this code assumes that you want to capture the payment immediately. If you want to authorize the payment and capture it later, you would change the `"intent"` to `"AUTHORIZE"`.

Lastly, ensure you have the necessary dependencies to compile and run this code, such as the org.json library to handle JSON objects.